In [79]:
from datetime import datetime
import sys
import time
import sqlite3
import pandas as pd
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 
import gensim
import re
from tika import parser
import spacy

In [ ]:
def initialize_db(db_name):
    connection = sqlite3.connect(db_name)
    connection.row_factory = sqlite3.Row

    cur = connection.cursor()
    cur.execute("SELECT * FROM Jobs")

    return connection, cur

def parse_job(row):
    return {
        'organization' : row['organization'],
        'job_title' : row['job_title'],
        'number_of_openings' : row['number_of_openings'],
        'address_line1' : row['address_line1'],
        'city' : row['city'],
        'province_state' : row['province_state'],
        'postal_code' : row['postal_code'],
        'job_summary' : row['job_summary'],
        'job_responsibilities' : row['job_responsibilities'],
        'required_skills' : row['required_skills'],
        'targeted_disciplines' : row['targeted_disciplines'].replace('\r', '').replace('\t', ' ').replace('\n', '').replace('Targeted Clusters', '')
    }

In [4]:
connection, cur = initialize_db("data/fall2017/jobs.db")

OperationalError: unable to open database file

In [ ]:
count = 0
jobs = []
while True:
    count += 1
    row = cur.fetchone()

    if row is None:
        print("========================== All records processed ============================")
        break

    job = parse_job(row)
    
    jobs.append(job)
    
print("Number of rows loaded: ", count)
connection.close()

In [ ]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(jobs, orient="columns")

In [ ]:
df.to_parquet('data/data.parquet', compression="gzip", engine='fastparquet')

In [7]:
#load
data = pd.read_parquet("data.parquet", engine="fastparquet")

In [8]:
print(data["organization"].values[971])
print(data["job_title"].values[971])
print(data["required_skills"].values[971])
print(len(data))

Nuance Communications Inc
Customer Records and Entitlement Co-Op (Brooks)
Knowledge, Skills and Qualifications:
-              Strong, structured analytical skills
-              Excellent time/project management skills
-              Strong and effective English language communications: verbal, listening, written and presentation skills.
-              Strong interpersonal skills
-              Exceptional customer service skills with effective follow-through
-              Willingness to own problems and see them to completion
-              Ability to work under pressure in a fast-paced production environment.
-              Excellent keyboard/data entry skills (speed & accuracy)
-              Self-starter and able to work in an unsupervised environment
Education:          Enrolled as an undergraduate or graduate student at an accredited college, university or technical/vocational institution.
Have a minimum GPA of 3.0
3932


In [9]:
def get_cat(row):
    if ("ENG - Software Engineering" in row["targeted_disciplines"]) and ("MATH - Computer Science" in row["targeted_disciplines"]):
        return True
    else:
        return False

In [10]:
def merge_duplicates(df):
    df = df.drop_duplicates(subset=['organization', 'job_title'], keep="first") 
    df = df.drop_duplicates(subset=['job_responsibilities', 'job_summary', 'required_skills'], keep="first")
    return df

In [11]:
#Used for Basic Keywords
def clean(x):
    newstr = strip_multiple_whitespaces(x)
    newstr = re.sub(r'[^\w]', "", newstr)
    return newstr


In [12]:
#Clean data
data["job_responsibilities"] = data["job_responsibilities"].map(lambda x: clean(x))
data["job_summary"] = data["job_summary"].map(lambda x: clean(x))
data["required_skills"] = data["required_skills"].map(lambda x: clean(x))
data = merge_duplicates(data)

In [13]:
m = data.apply(get_cat, axis=1)
data = data[m]

In [14]:
data.head()

,address_line1,city,job_responsibilities,job_summary,job_title,number_of_openings,organization,postal_code,province_state,required_skills,targeted_disciplines
1,187 King Street South,Waterloo,Responsibilities -Under the direction of staff...,"Validus Research Inc., based in Waterloo, Onta...",Developer,1,Validus Group,N2J 1R1,Ontario,Essential Skills/Qualifications -Undergraduate...,- Theme - Business Administratio...
11,"144 Front Street West, Suite 685",Toronto,Peraso's Product Validation engineers are resp...,Company Profile Peraso is a fabless semiconduc...,Validation Engineering,1,Peraso Technologies Inc,M5J 2L7,Ontario,Minimum Qualifications: - Understanding of C/C...,- Theme - Computing: Information...
12,67 Yonge Street,Toronto,We are looking for a QA Automation Developer t...,"QuickTapSurvey, created by TabbleDabble helps ...",Automation Developer,1,QuickTapSurvey created by TabbleDabble Inc.,M5E 1J8,Ontario,You must have development skills that consist ...,- Theme - Computing: Information...
14,609 Kumpf Drive,Waterloo,- Build strong partnerships with software engi...,As part of the Global R&D; Software organizati...,Technical Writer,1,DEMATIC,N2V 1K8,Ontario,- Exceptional writing skills - Proficiency wit...,- Theme - Computing: Information...
15,,,RESPONSIBILITIES Review and develop understand...,Are you looking for the opportunity to be invo...,Manufacturing Software Development,1,Ford Motor Company of Canada Limited,,Ontario,ESSENTIAL SKILLS AND QUALIFICATIONS Working to...,- Theme - Computing: Hardware ...


In [51]:
#create corpus
def preprocess(x):
    x = re.sub(r"\S*@\S*\s?", "", x) #remove emails
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x) #remove website and urls
    
    x = gensim.utils.simple_preprocess(x)
    x = " ".join(x)
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords]
    x = preprocess_string(x, CUSTOM_FILTERS)
    return x
    
def read_corpus():
    for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
        doc = doc1 + " " + doc2 + " " + doc3
        yield gensim.models.doc2vec.TaggedDocument(preprocess(doc), [i]) #formerly gensim.utils.simple_preprocess


In [53]:
train_corpus = list(read_corpus())

In [54]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=75)

In [55]:
model.build_vocab(train_corpus)

In [56]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 39.9 s


In [111]:
#parse resume
raw = parser.from_file('ResumeKevinShen.pdf')
print(raw['content'])
























 

Kevin Shen 
+1 647-994-7088 | kshen3778@gmail.com |​ ​linkedin.com/in/kevinkwshen​ ​|​ ​github.com/kshen3778 

 

Education  

University of Waterloo​, Bachelor of Computer Science Co-op                      2018 -  2023 (expected) 

The Knowledge Society  February 2017 - May 2017 

 

Professional Experience 

Kinaxis     April 2018 - August 2018 
Machine Learning Intern 

● Created and presented a patent-pending machine learning prototype to the CEO and senior executives. 
● Trained and evaluated time series forecasting models such as DeepAR and LightGBM on customer data. 
● Mentored interns on projects that involved sentiment analysis, anomaly detection, and data preprocessing. 

 
Omicron       October 2017 - Present 
Co-founder, Lead Developer  

● Created fully functional platform to collect hackathon data for analytics (launched at a ​150​ people hackathon). 
● Secured over​ $17k+ ​worth of sponsorship for over ​8​ events (​1200+​ attendees combined) si

In [67]:
preprocess("● Created and presented a patent-pending machine learning prototype to the CEO and senior executives. ● Trained and evaluated time series forecasting models such as DeepAR and LightGBM on customer data. ● Mentored interns on projects that involved sentiment analysis, anomaly detection, and data preprocessing." )

['created',
 'presented',
 'patent',
 'pending',
 'machine',
 'learning',
 'prototype',
 'ceo',
 'senior',
 'executives',
 'trained',
 'evaluated',
 'time',
 'series',
 'forecasting',
 'models',
 'deepar',
 'lightgbm',
 'customer',
 'data',
 'mentored',
 'interns',
 'projects',
 'involved',
 'sentiment',
 'analysis',
 'anomaly',
 'detection',
 'data',
 'preprocessing']

In [80]:
nlp = spacy.load('en_core_web_sm')

In [117]:
category = "data science"

resume = "Worked on proof-of-concept projects to demonstrate ML feasibility in key supply chain areas. \
Pre-processed time series data, trained forecasting models, and evaluated predictions. \
Mentored other interns on data science libraries, Jupyter, natural language processing, and ML best practices."
# resume = "Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. \
# Created fully functional platform to collect hackathon data for analytics. \
# Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan "
# resume = "Experimented with state of the art NLP techniques to detect type of toxicity in online comments. \
# Experimented with LSTMs, word embeddings (Glove, FastText), and techniques such as sequence bucketing. \
# Fine-tuned and ensembled Bidirectional Encoder Representations from Transformers (BERT). "
# Integrated an “auto-swipe” feature into Tinder based on real-time brain EEG data collected by the Muse. \
# Tested on participants at Hack The 6ix 2017 and achieved a 75% accuracy. \
# Knowledgeable in: Git, Docker, Python (Pandas, Numpy, Scikit-Learn), Keras, Pytorch, Java, Javascript, VueJS, Firebase \
# "



tagged = nlp(clean(raw["content"])) #THIS IS NOT GOOD, FIND SOME OTHER WAY TO GET KEYWORDS
keywords = []

categories = dict()
categories["data science"] = [" data science", " machine learning", " artificial intelligence", " data engineering", " AI", " ML"]

for token in tagged:
    if token.pos_ == "PROPN":
        keywords.append(token.text)

keywords += categories[category]
keywords = list(set(keywords))

print(keywords)

resume_processed = preprocess(raw["content"])
print(resume_processed)

['Unintended', '|', 'Network', 'GAN', 'Feature', 'Top', 'Coinberry', 'Images', ' data science', 'Fine', 'Shen', 'MongoDB', 'Projects:\u200b', 'Keras', '\u200bHack', 'Notebook', 'Professional', 'August', 'University', 'of\u200b', 'Co', 'Bias', 'Spacy', '\u200b1200+\u200b', 'Numpy', 'May', 'October', 'Jigsaw', 'North', ' artificial intelligence', '\u200bPandas', 'label', 'Hackathon', 'Bezant', 'Bachelor', 'DeepAR', 'Kinaxis', 'kshen3778', 'Classification', 'Python', 'Bronze', 'Machine', '’s', 'Sapien', 'Winner\u200b', 'Intern', 'EEG', 'Lead', 'Networks', 'Learning', 'Medal', 'February', 'Ingredient', '\u200bredbull.com', 'million\u200b', 'Matching', 'Computer', 'Kaggle', 'Java', 'Language', 'Kevin', 'NLP', 'kevinkwshen\u200b', 'Jupyter', 'Pytorch', 'Muse', 'Learn\u200b', 'Gensim', 'Red', 'Firebase', 'Toxicity', 'Musr', 'Multi', ' machine learning', '\u200b0.94187\u200b.', 'Science', 'Education', 'VueJS', 'April', 'Scikit', 'Mentored', '6ix\u200b', 'Experience', 'Ingredients', 'Docker', '

In [118]:
#Get keyword match score
def score(keywords, job):
    score = 0
    for word in keywords:
        if word in job:
            score += 1
    return score/len(keywords)

In [119]:
inferred_vector = model.infer_vector(resume_processed)
sims = model.docvecs.most_similar([inferred_vector], topn=len(data))

In [120]:
sims

[(949, 0.2921101748943329),
 (1079, 0.2611527442932129),
 (478, 0.26060551404953003),
 (1056, 0.2479456514120102),
 (1258, 0.24734464287757874),
 (744, 0.24629104137420654),
 (399, 0.23886142671108246),
 (1133, 0.23746807873249054),
 (429, 0.23313312232494354),
 (883, 0.2318217009305954),
 (246, 0.23123744130134583),
 (95, 0.2213352918624878),
 (199, 0.2192341387271881),
 (732, 0.21863016486167908),
 (290, 0.21706059575080872),
 (149, 0.21680980920791626),
 (303, 0.21508747339248657),
 (216, 0.2150336503982544),
 (742, 0.21244023740291595),
 (920, 0.21209748089313507),
 (180, 0.2095929980278015),
 (1003, 0.20706050097942352),
 (1006, 0.20581477880477905),
 (403, 0.2050999402999878),
 (1232, 0.20406633615493774),
 (24, 0.20112434029579163),
 (188, 0.20070885121822357),
 (751, 0.19957205653190613),
 (611, 0.19951406121253967),
 (466, 0.1989765465259552),
 (455, 0.1975068598985672),
 (160, 0.19714316725730896),
 (1131, 0.1970798522233963),
 (649, 0.19363798201084137),
 (42, 0.192204371094

In [121]:
cosine_ranking = dict(sims)
cosine_ranking

{949: 0.2921101748943329,
 1079: 0.2611527442932129,
 478: 0.26060551404953003,
 1056: 0.2479456514120102,
 1258: 0.24734464287757874,
 744: 0.24629104137420654,
 399: 0.23886142671108246,
 1133: 0.23746807873249054,
 429: 0.23313312232494354,
 883: 0.2318217009305954,
 246: 0.23123744130134583,
 95: 0.2213352918624878,
 199: 0.2192341387271881,
 732: 0.21863016486167908,
 290: 0.21706059575080872,
 149: 0.21680980920791626,
 303: 0.21508747339248657,
 216: 0.2150336503982544,
 742: 0.21244023740291595,
 920: 0.21209748089313507,
 180: 0.2095929980278015,
 1003: 0.20706050097942352,
 1006: 0.20581477880477905,
 403: 0.2050999402999878,
 1232: 0.20406633615493774,
 24: 0.20112434029579163,
 188: 0.20070885121822357,
 751: 0.19957205653190613,
 611: 0.19951406121253967,
 466: 0.1989765465259552,
 455: 0.1975068598985672,
 160: 0.19714316725730896,
 1131: 0.1970798522233963,
 649: 0.19363798201084137,
 42: 0.19220437109470367,
 335: 0.19110217690467834,
 962: 0.19082212448120117,
 376: 0.

In [122]:
#get keyword + cosine score
start = time.time()
scores = []
cosine_weight = 0.5
keyword_weight = 0.5
for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
    doc = doc1 + " " + doc2 + " " + doc3
    keyword_score = score(keywords, doc)
    cosine_score = cosine_ranking[i]
    final_score = cosine_weight * cosine_score + keyword_weight * keyword_score
    scores.append((i, final_score))
print(time.time() - start)

0.6750340461730957


In [123]:
sorted_scores = sorted(scores,key=lambda x: x[1], reverse=True)

In [124]:
sorted_scores

[(478, 0.16718800292640434),
 (949, 0.16244853007011725),
 (1258, 0.1605575673404287),
 (1079, 0.15516653608103267),
 (883, 0.14869773571119935),
 (1056, 0.14446462898469362),
 (199, 0.1424039546094957),
 (926, 0.1416064886773219),
 (539, 0.13995267609592343),
 (991, 0.13634739694048148),
 (744, 0.1354406026543164),
 (1133, 0.13512748198919608),
 (1232, 0.13482005332337052),
 (732, 0.13390524636526577),
 (290, 0.1331204618098306),
 (429, 0.13296000378542258),
 (399, 0.13172579532275436),
 (1131, 0.1313268113575998),
 (403, 0.13123849474015784),
 (932, 0.12903494803143328),
 (149, 0.12889670788264665),
 (962, 0.12819794748650223),
 (95, 0.1270610885541947),
 (920, 0.12654054372525605),
 (180, 0.12528830229258928),
 (246, 0.12381544196214832),
 (1006, 0.12339919268107805),
 (1012, 0.12322914527087915),
 (742, 0.1226135613244088),
 (350, 0.12248688606453723),
 (466, 0.11998007654166612),
 (605, 0.1199779007278505),
 (303, 0.1198388186634564),
 (216, 0.11981190716634031),
 (376, 0.11963950

In [125]:
for job in sorted_scores[:50]:
    X = job[0]
    print("ID: ", X)
    print(data["job_title"].values[X], ", ", data["organization"].values[X])
    print()
    print(data["job_responsibilities"].values[X])
    print()
    print(data["job_summary"].values[X])
    print()
    print(data["required_skills"].values[X])
    print("================================")
    print()

ID:  478
Full Stack Developer ,  360insights

Participate in developing 360insights' industry leading incentive management SaaS platform Contribute to all aspects of the software lifecycle from feature inception and design to test design and continuous code improvement Actively participate in a high performance team including pair programming with the latest JavaScript technologies and code reviews with senior developers You will deploy and support code in production systems

360insights office is a 10-15 minute walk from the GO station. At 360insights, our culture is driven by a simple concept - happiness. We believe happiness makes for better business and a better world and we know it makes a difference in how our team serves our clients. Awarded "Best Workplace in Canada" by the Great Place to Work® Institute, and rocking a Net Promoter Score of 88%, 360insights is redefining the client experience, and having a ton of fun in the process! We are seeking a Full Stack Developer to join

We are looking for amazing people who have a passion in coding and have a good head on their shoulders. We want web and app developers who are talented and smart people with a solid foundation of coding languages and problem solving. We have an opening for full-time Software Developers in the following roles: web and/or mobile development, front-end, back-end and full stack. Our Previous & Current PEY's Highlights: -Seeing their code in action in our final product -Participating in a Go lang boot-camp taught by the leading Go lang instructor William Kennedy -Being a part of the actual production of our product that is used by all of Canada, the US, and the UK -Being seen as a part of the Dev. team and not just an 'intern' -Surrounded by amazing and talented Devs who guide you

Think you have what it takes? Here's what we're looking for: - You are currently enrolled in a software engineering-related discipline, preferably in Computer Science. - Experience developing web/ mobile applicat

The following are must-have requirements for the position: - At least 1 year of a B.S. degree in Software/Computer/Electrical Engineering, Computer Science or similar field - Strong problem solving and analytical skills - Exceptionally bright and talented with a passion for learning and software development - Great team player who is willing to help out wherever and whenever they can - A desire to learn about software testing In addition, experience in one or more of the following areas is desirable: - Experience with messaging, such as JMS, MQ or other messaging middleware systems - Knowledge of network programming and protocols (e.g. TCP/IP, Berkley sockets interface) - Object-oriented programming skills - A strong background and understanding of networking protocols and concepts or network management is an asset - Familiarization with multi-threaded applications and concurrent programming techniques - Experience with scripting languages and applying them to increase the productivity

We are looking to add data scientists and computer engineers to our rapidly growing machine learning team. We will only consider candidates that are genuinely interested in staying in Canada upon graduation. As a machine learning intern, you will: - Research, develop, and apply new techniques in the intersection of deep learning and personalization to further advance our industry leading product. - Work with diverse real-life datasets that range from banking transactions, to unstructured text, to images and audio streams. - Collaborate closely with the engineering team in a fast paced startup environment and see your research deployed in production with very short turnaround.

You: 4A or 3B software engineering, computer engineering, systems design, CS or math student with an interest in a career in Machine Learning in Toronto upon graduation. Us: Layer 6 AI (layer6.ai), Toronto's largest AI company, with expertise in delivering Deep Learning frameworks to global companies. The team co

As a Software Developer you will work with the other members of the Development Team and will be assigned new feature development projects. Initially we will start with a small project to get you familiar with our technology stack and provide opportunity for you to learn more on the layers you haven't worked with before. This will expand to bigger, more complex projects based on how the ramp up period goes. This role will focus primarily on working with C# / .NET so prior experience is a plus. Past students have accomplished: - Realtime, distributed, multi-display web-based dashboard management system, using C#, React over Websockets - Implementation and rollout of system wide single-sign-on across more than five platforms - Portal for managing searches across our entire AWS server farm - Adapted front-end React to work with Google Analytics platform for system usage tracking

Shoplogix is part of the family of companies that belong to Constellation Software which is the largest softwa

...

Product Integrity ,  Apple Inc

In Product Integrity, we strongly believe that the best co-ops are a two-way street. As a result, the specific projects you work on are selected based on your skills and interests. In addition to your product work, you will work together with your manager on a larger scale project designed to highlight your skills. These visionary projects typically have high impact and are frequently presented to the VP of Product Integrity. It is a great chance for co-ops to showcase a blend of skills, passion, and drive to senior leadership at Apple. Successful Product Integrity candidates will be able to demonstrate: - A strong technical foundation to help build Hardware and/or Software tools - A track record of taking a leadership role in personal, school or work projects - Some knowledge of Quality concepts and terminology (white box, black box, test harness, etc.) - Analytical skills to assist in risk analysis of defects - Ability to solve difficult technical prob

The Chan Zuckerberg Initiative is dedicated to advancing human potential and promoting equal opportunity through technology, grantmaking, impact investing, policy, and advocacy work. We look for bold ideas - regardless of structure and stage - and help them scale by pairing world-class engineers with subject matter experts to build tools that accelerate the pace of social progress. WHAT WE BELIEVE We engage directly in the communities we serve because no one understands how to address our society's challenges better than those who live them everyday. These partners help us identify problems and opportunities, learn fast and iterate toward our audacious goals for the next century. We strive to be humble, empathetic and scrappy life-long learners who work collaboratively to develop community-centered solutions. We believe that high-performing teams include people from different backgrounds and experiences who can challenge each other's assumptions with fresh perspectives. To that end, th


ID:  751
3D Software Developer ,  Side Effects Software Inc

Students will be given a complete project to work on based on their interests and level of experience. Projects can be in any area of the software such as computational geometry, physically-based rendering, character rigging and animation, physical simulation, algorithmic optimization, GPU programming, parallel/distributed computing, real-time engines, language/compiler design and optimization, 2D and 3D user interfaces, and more.

Side Effects Software is looking for several outstanding co-op students to join its R&D; team. Ideal candidates will have superior knowledge of C++ programming, a passion for graphics, and a record of academic excellence. Side Effects Software has been in the computer graphics business for almost 25 years. For more than a decade, our flagship product has been Houdini, an award-winning software designed for high-end visual effects and 3D animation for film, commercials, and video games. Studios usi

Coursera's mission is to transform lives by providing access to the world's best education. Our engineering team uses a modern tech stack, building all of Coursera on top of AWS. Our website uses React and Webpack written all in ES6, we use Swift for iOS and Java for Android and our backend is built in Scala using MySQL and Cassandra as a datastore. We are looking for rising sophomores and juniors for our spring, summer and winter internship programs. We typically have a dozen interns each program. The programs are highly competitive, so apply as soon as you know you are available. The growth team is looking for their next intern! This team leverages analytics to understand where users are and what courses they are looking for. Through SEO, onboarding, search, discovery, recommendations, site performance and purchase flows, the growth team converts users to learners on a daily basis. One small change on this team can lead to thousands of new learners a day.

Pursuing a degree in Comput

You will have: - Successfully completed at least two years in a recognized university level: Computer Science, Computer Engineering, Electrical Engineering or Math program - Software development experience through course work, hobbies or previous work placements in industry - A passion for C++ programming and the benefits of using templated libraries to create high quality, robust software - Demonstrated experience in object oriented design and implementation, including experience with multi-threaded and asynchronous environments - Experience with standard software development methodologies Development experience with any of the following would be considered an asset: - H.264 video encoding/decoding - Open source audio/video frameworks (x264 and ffmpeg) - UDP/socket distributed network programming using boost ASIO - Cross-platform C/C++ development (Windows, Linux, OSX, iOS, Android) - Wireless network development experience (HSPA, LTE, WiMAX) - Linux iptables, routing, bridging, tun/t